In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [8]:
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import ADASYN
from sklearn.cluster import KMeans

ada= ADASYN(random_state= 15)
cc= ClusterCentroids(random_state= 15, estimator= KMeans(n_init= 10))

X_ada, y_ada= ada.fit_resample(X_train, y_train)
X_ada_cc, y_ada_cc= cc.fit_resample(X_ada, y_ada)

--- LogisticRegression ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr_ada_cc= LogisticRegression(random_state=15)
lr_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_lr_ada_cc= lr_ada_cc.predict(X_test)
y_pred_prob_lr_ada_cc= lr_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lr_ada_cc, labels= lr_ada_cc.classes_)
print(classification_report(y_test, y_pred_lr_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_lr_ada_cc))

              precision    recall  f1-score   support

           0       0.97      0.73      0.83      8517
           1       0.24      0.82      0.38       912

    accuracy                           0.74      9429
   macro avg       0.61      0.77      0.60      9429
weighted avg       0.90      0.74      0.79      9429

Precision: 0.24387855044074436
Accuracy: 0.7368755965637925
AUC: 0.7735761063013292


--- DecisionTreeClassifier ---

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt_ada_cc= DecisionTreeClassifier(random_state=15)
dt_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_dt_ada_cc= dt_ada_cc.predict(X_test)
y_pred_prob_dt_ada_cc= dt_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_dt_ada_cc, labels= dt_ada_cc.classes_)
print(classification_report(y_test, y_pred_dt_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt_ada_cc))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      8517
           1       0.23      0.25      0.24       912

    accuracy                           0.85      9429
   macro avg       0.58      0.58      0.58      9429
weighted avg       0.85      0.85      0.85      9429

Precision: 0.23302938196555217
Accuracy: 0.8473857248912928
AUC: 0.5810852495215967


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt_ada_cc= GridSearchCV(estimator= dt_ada_cc, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt_ada_cc.fit(X_ada_cc, y_ada_cc)

print("Best Parameters:", gs_dt_ada_cc.best_params_)
print("Best Precision Score:", gs_dt_ada_cc.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 30, 'max_features': None, 'min_samples_leaf': 10, 'min_samples_split': 2}
Best Precision Score: 0.9426531201437577


In [12]:
tuned_dt_ada_cc= gs_dt_ada_cc.best_estimator_
y_pred_tuned_dt_ada_cc= tuned_dt_ada_cc.predict(X_test)
y_pred_prob_tuned_dt_ada_cc= tuned_dt_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt_ada_cc, labels= tuned_dt_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_dt_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt_ada_cc))

              precision    recall  f1-score   support

           0       0.92      0.95      0.93      8517
           1       0.29      0.20      0.24       912

    accuracy                           0.88      9429
   macro avg       0.61      0.57      0.58      9429
weighted avg       0.86      0.88      0.87      9429

Precision: 0.2944078947368421
Accuracy: 0.8767631774313289
AUC: 0.7015535492482528


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn_ada_cc= KNeighborsClassifier()
knn_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_knn_ada_cc= knn_ada_cc.predict(X_test)
y_pred_prob_knn_ada_cc= knn_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_knn_ada_cc, labels= knn_ada_cc.classes_)
print(classification_report(y_test, y_pred_knn_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn_ada_cc))

              precision    recall  f1-score   support

           0       0.95      0.77      0.85      8517
           1       0.22      0.60      0.32       912

    accuracy                           0.75      9429
   macro avg       0.58      0.69      0.58      9429
weighted avg       0.88      0.75      0.80      9429

Precision: 0.21696252465483234
Accuracy: 0.7510870717997666
AUC: 0.7393456121812103


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn_ada_cc= GridSearchCV(estimator= knn_ada_cc, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn_ada_cc.fit(X_ada_cc, y_ada_cc)
print("Best Parameters:", gs_knn_ada_cc.best_params_)
print("Best Precision Score:", gs_knn_ada_cc.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
Best Precision Score: 0.9288368335424606


In [15]:
tuned_knn_ada_cc= gs_knn_ada_cc.best_estimator_
y_pred_tuned_knn_ada_cc= tuned_knn_ada_cc.predict(X_test)
y_pred_prob_tuned_knn_ada_cc= tuned_knn_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn_ada_cc, labels= tuned_knn_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_knn_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn_ada_cc))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      8517
           1       0.25      0.19      0.21       912

    accuracy                           0.87      9429
   macro avg       0.58      0.56      0.57      9429
weighted avg       0.85      0.87      0.86      9429

Precision: 0.24643874643874644
Accuracy: 0.8655212641849613
AUC: 0.62076813864531


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb_ada_cc= GaussianNB()
nb_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_nb_ada_cc= nb_ada_cc.predict(X_test)
y_pred_prob_nb_ada_cc= nb_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_nb_ada_cc, labels= nb_ada_cc.classes_)
print(classification_report(y_test, y_pred_nb_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb_ada_cc))

              precision    recall  f1-score   support

           0       0.98      0.53      0.68      8517
           1       0.17      0.89      0.28       912

    accuracy                           0.56      9429
   macro avg       0.57      0.71      0.48      9429
weighted avg       0.90      0.56      0.64      9429

Precision: 0.16666666666666666
Accuracy: 0.5605048255382331
AUC: 0.8012693652941794


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc_ada_cc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_svc_ada_cc= svc_ada_cc.predict(X_test)
y_pred_prob_svc_ada_cc= svc_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_svc_ada_cc, labels= svc_ada_cc.classes_)
print(classification_report(y_test, y_pred_svc_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc_ada_cc))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      8517
           1       0.16      0.04      0.06       912

    accuracy                           0.89      9429
   macro avg       0.53      0.51      0.50      9429
weighted avg       0.83      0.89      0.85      9429

Precision: 0.15695067264573992
Accuracy: 0.8870505886096086
AUC: 0.6990053690348921


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf_ada_cc= RandomForestClassifier(random_state=15)
rf_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_rf_ada_cc= rf_ada_cc.predict(X_test)
y_pred_prob_rf_ada_cc= rf_ada_cc.predict_proba(X_test)[:,1]

In [19]:
cm= confusion_matrix(y_test, y_pred_rf_ada_cc, labels= rf_ada_cc.classes_)
print(classification_report(y_test, y_pred_rf_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf_ada_cc))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      8517
           1       0.38      0.11      0.17       912

    accuracy                           0.90      9429
   macro avg       0.64      0.55      0.56      9429
weighted avg       0.86      0.90      0.87      9429

Precision: 0.375
Accuracy: 0.8960653303637713
AUC: 0.8212884087346461


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf_ada_cc= GridSearchCV(estimator= rf_ada_cc, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf_ada_cc.fit(X_ada_cc, y_ada_cc)
print("Best Parameters:", gs_tuned_rf_ada_cc.best_params_)
print("Best Precision Score:", gs_tuned_rf_ada_cc.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}
Best Precision Score: 0.9779167589795433


In [21]:
tuned_rf_ada_cc= gs_tuned_rf_ada_cc.best_estimator_
y_pred_tuned_rf_ada_cc= tuned_rf_ada_cc.predict(X_test)
y_pred_prob_tuned_rf_ada_cc= tuned_rf_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf_ada_cc, labels= tuned_rf_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_rf_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf_ada_cc))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95      8517
           1       0.44      0.12      0.19       912

    accuracy                           0.90      9429
   macro avg       0.67      0.55      0.57      9429
weighted avg       0.87      0.90      0.87      9429

Precision: 0.436
Accuracy: 0.8998833386361226
AUC: 0.8317986704609358


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada_ada_cc= AdaBoostClassifier(random_state=15)
ada_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_ada_ada_cc= ada_ada_cc.predict(X_test)
y_pred_prob_ada_ada_cc= ada_ada_cc.predict_proba(X_test)[:,1]

In [23]:
cm= confusion_matrix(y_test, y_pred_ada_ada_cc, labels= ada_ada_cc.classes_)
print(classification_report(y_test, y_pred_ada_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada_ada_cc))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93      8517
           1       0.39      0.42      0.40       912

    accuracy                           0.88      9429
   macro avg       0.66      0.67      0.67      9429
weighted avg       0.88      0.88      0.88      9429

Precision: 0.389917695473251
Accuracy: 0.8805811857036802
AUC: 0.8399053930323048


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada_ada_cc= GridSearchCV(estimator= ada_ada_cc, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada_ada_cc.fit(X_ada_cc, y_ada_cc)
print("Best Parameters:", gs_ada_ada_cc.best_params_)
print("Best Precision Score:", gs_ada_ada_cc.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.9657468667095896


In [25]:
tuned_ada_ada_cc= gs_ada_ada_cc.best_estimator_
y_pred_tuned_ada_ada_cc= tuned_ada_ada_cc.predict(X_test)
y_pred_prob_tuned_ada_ada_cc= tuned_ada_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada_ada_cc, labels= tuned_ada_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_ada_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada_ada_cc))

              precision    recall  f1-score   support

           0       0.92      0.97      0.95      8517
           1       0.48      0.24      0.32       912

    accuracy                           0.90      9429
   macro avg       0.70      0.61      0.63      9429
weighted avg       0.88      0.90      0.89      9429

Precision: 0.47649572649572647
Accuracy: 0.9009438964895535
AUC: 0.8494423047609632


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb_ada_cc= GradientBoostingClassifier(random_state=15)
grb_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_grb_ada_cc= grb_ada_cc.predict(X_test)
y_pred_prob_grb_ada_cc= grb_ada_cc.predict_proba(X_test)[:,1]


cm= confusion_matrix(y_test, y_pred_grb_ada_cc, labels= grb_ada_cc.classes_)
print(classification_report(y_test, y_pred_grb_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb_ada_cc))

              precision    recall  f1-score   support

           0       0.93      0.96      0.94      8517
           1       0.42      0.29      0.34       912

    accuracy                           0.89      9429
   macro avg       0.67      0.62      0.64      9429
weighted avg       0.88      0.89      0.88      9429

Precision: 0.41601255886970173
Accuracy: 0.8919291547353908
AUC: 0.8433746542003713


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb_ada_cc= GridSearchCV(estimator= grb_ada_cc, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb_ada_cc.fit(X_ada_cc, y_ada_cc)

print("Best Parameters:", gs_grb_ada_cc.best_params_)
print("Best Precision Score:", gs_grb_ada_cc.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.9776058834631203


In [28]:
tuned_grb_ada_cc= gs_grb_ada_cc.best_estimator_
y_pred_tuned_grb_ada_cc= tuned_grb_ada_cc.predict(X_test)
y_pred_prob_tuned_grb_ada_cc= tuned_grb_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb_ada_cc, labels= tuned_grb_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_grb_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb_ada_cc))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.53      0.19      0.27       912

    accuracy                           0.91      9429
   macro avg       0.72      0.58      0.61      9429
weighted avg       0.88      0.91      0.88      9429

Precision: 0.528125
Accuracy: 0.9051861279032771
AUC: 0.8487080920717903


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb_ada_cc= XGBClassifier(random_state=15)
xgb_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_xgb_ada_cc= xgb_ada_cc.predict(X_test)
y_pred_prob_xgb_ada_cc= xgb_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb_ada_cc, labels= xgb_ada_cc.classes_)
print(classification_report(y_test, y_pred_xgb_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb_ada_cc))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95      8517
           1       0.46      0.13      0.20       912

    accuracy                           0.90      9429
   macro avg       0.69      0.56      0.57      9429
weighted avg       0.87      0.90      0.88      9429

Precision: 0.4573643410852713
Accuracy: 0.9009438964895535
AUC: 0.835753158286111


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb_ada_cc= GridSearchCV(estimator= xgb_ada_cc, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb_ada_cc.fit(X_ada_cc, y_ada_cc)

print("Best Parameters:", gs_xgb_ada_cc.best_params_)
print("Best Precision Score:", gs_xgb_ada_cc.best_score_)

tuned_xgb_ada_cc= gs_xgb_ada_cc.best_estimator_
y_pred_tuned_xgb_ada_cc= tuned_xgb_ada_cc.predict(X_test)
y_pred_prob_tuned_xgb_ada_cc= tuned_xgb_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb_ada_cc, labels= tuned_xgb_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_xgb_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb_ada_cc))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.9753656441201602
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.52      0.18      0.27       912

    accuracy                           0.90      9429
   macro avg       0.72      0.58      0.61      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.5154320987654321
Accuracy: 0.9043376816205324
AUC: 0.8461194226613853


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm_ada_cc= LGBMClassifier(random_state=15)
lgm_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_lgm_ada_cc= lgm_ada_cc.predict(X_test)
y_pred_prob_lgm_ada_cc= lgm_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm_ada_cc, labels= lgm_ada_cc.classes_)
print(classification_report(y_test, y_pred_lgm_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm_ada_cc))

[LightGBM] [Info] Number of positive: 34065, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006942 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8603
[LightGBM] [Info] Number of data points in the train set: 68130, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.55      0.13      0.22       912

    accuracy                           0.91      9429
   macro avg       0.73      0.56      0.58      9429
weighted avg       0.88      0.91      0.88      9429

Precision: 0.5466666666666666
Accuracy: 0.9055042952593064
AUC: 0.8457723999884648


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm_ada_cc= GridSearchCV(estimator= lgm_ada_cc, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm_ada_cc.fit(X_ada_cc, y_ada_cc)

print("Best Parameters:", gs_lgm_ada_cc.best_params_)
print("Best Precision Score:", gs_lgm_ada_cc.best_score_)

tuned_lgm_ada_cc= gs_lgm_ada_cc.best_estimator_
y_pred_tuned_lgm_ada_cc= tuned_lgm_ada_cc.predict(X_test)
y_pred_prob_tuned_lgm_ada_cc= tuned_lgm_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm_ada_cc, labels= tuned_lgm_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_lgm_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm_ada_cc))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Info] Number of positive: 34065, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008790 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8603
[LightGBM] [Info] Number of data points in the train set: 68130, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.1, 'min_data_in_leaf': 50, 'num_leaves': 31, 'reg_alpha': 0.5}
Best Precision Score: 0.9848597807765044
[LightGBM] [Warning] min_data_in_leaf is set=50, m

--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat_ada_cc= CatBoostClassifier(random_state=15)
cat_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_cat_ada_cc= cat_ada_cc.predict(X_test)
y_pred_prob_cat_ada_cc= cat_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_cat_ada_cc, labels= cat_ada_cc.classes_)
print(classification_report(y_test, y_pred_cat_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat_ada_cc))

Learning rate set to 0.062484
0:	learn: 0.6233949	total: 172ms	remaining: 2m 51s
1:	learn: 0.5693385	total: 206ms	remaining: 1m 42s
2:	learn: 0.5240582	total: 235ms	remaining: 1m 18s
3:	learn: 0.4795593	total: 258ms	remaining: 1m 4s
4:	learn: 0.4519124	total: 279ms	remaining: 55.6s
5:	learn: 0.4275149	total: 298ms	remaining: 49.4s
6:	learn: 0.4030914	total: 315ms	remaining: 44.6s
7:	learn: 0.3832487	total: 331ms	remaining: 41s
8:	learn: 0.3682250	total: 347ms	remaining: 38.2s
9:	learn: 0.3561071	total: 362ms	remaining: 35.9s
10:	learn: 0.3411275	total: 379ms	remaining: 34s
11:	learn: 0.3294933	total: 395ms	remaining: 32.5s
12:	learn: 0.3190637	total: 415ms	remaining: 31.5s
13:	learn: 0.3062144	total: 436ms	remaining: 30.7s
14:	learn: 0.2995158	total: 452ms	remaining: 29.7s
15:	learn: 0.2939441	total: 469ms	remaining: 28.8s
16:	learn: 0.2884186	total: 486ms	remaining: 28.1s
17:	learn: 0.2778442	total: 502ms	remaining: 27.4s
18:	learn: 0.2683227	total: 520ms	remaining: 26.9s
19:	learn: 0

162:	learn: 0.1492793	total: 4.61s	remaining: 23.7s
163:	learn: 0.1492051	total: 4.63s	remaining: 23.6s
164:	learn: 0.1491121	total: 4.65s	remaining: 23.5s
165:	learn: 0.1490401	total: 4.67s	remaining: 23.4s
166:	learn: 0.1489453	total: 4.68s	remaining: 23.4s
167:	learn: 0.1488434	total: 4.7s	remaining: 23.3s
168:	learn: 0.1486879	total: 4.72s	remaining: 23.2s
169:	learn: 0.1485998	total: 4.74s	remaining: 23.1s
170:	learn: 0.1484803	total: 4.76s	remaining: 23.1s
171:	learn: 0.1484087	total: 4.79s	remaining: 23s
172:	learn: 0.1482370	total: 4.82s	remaining: 23.1s
173:	learn: 0.1481306	total: 4.84s	remaining: 23s
174:	learn: 0.1480584	total: 4.86s	remaining: 22.9s
175:	learn: 0.1478489	total: 4.88s	remaining: 22.9s
176:	learn: 0.1477318	total: 4.9s	remaining: 22.8s
177:	learn: 0.1476399	total: 4.92s	remaining: 22.7s
178:	learn: 0.1475546	total: 4.94s	remaining: 22.6s
179:	learn: 0.1469331	total: 4.96s	remaining: 22.6s
180:	learn: 0.1468357	total: 4.98s	remaining: 22.5s
181:	learn: 0.1466

324:	learn: 0.1313415	total: 7.77s	remaining: 16.1s
325:	learn: 0.1311929	total: 7.79s	remaining: 16.1s
326:	learn: 0.1311385	total: 7.81s	remaining: 16.1s
327:	learn: 0.1310806	total: 7.83s	remaining: 16s
328:	learn: 0.1310237	total: 7.85s	remaining: 16s
329:	learn: 0.1309806	total: 7.87s	remaining: 16s
330:	learn: 0.1309237	total: 7.89s	remaining: 15.9s
331:	learn: 0.1308729	total: 7.9s	remaining: 15.9s
332:	learn: 0.1308232	total: 7.92s	remaining: 15.9s
333:	learn: 0.1307607	total: 7.94s	remaining: 15.8s
334:	learn: 0.1306995	total: 7.96s	remaining: 15.8s
335:	learn: 0.1306632	total: 7.98s	remaining: 15.8s
336:	learn: 0.1305681	total: 8s	remaining: 15.7s
337:	learn: 0.1303977	total: 8.02s	remaining: 15.7s
338:	learn: 0.1302478	total: 8.04s	remaining: 15.7s
339:	learn: 0.1301923	total: 8.06s	remaining: 15.6s
340:	learn: 0.1301385	total: 8.08s	remaining: 15.6s
341:	learn: 0.1300950	total: 8.1s	remaining: 15.6s
342:	learn: 0.1300360	total: 8.12s	remaining: 15.6s
343:	learn: 0.1299812	t

485:	learn: 0.1220435	total: 10.9s	remaining: 11.6s
486:	learn: 0.1219873	total: 11s	remaining: 11.5s
487:	learn: 0.1219343	total: 11s	remaining: 11.5s
488:	learn: 0.1218552	total: 11s	remaining: 11.5s
489:	learn: 0.1217951	total: 11s	remaining: 11.5s
490:	learn: 0.1217422	total: 11s	remaining: 11.4s
491:	learn: 0.1216999	total: 11s	remaining: 11.4s
492:	learn: 0.1216600	total: 11.1s	remaining: 11.4s
493:	learn: 0.1216132	total: 11.1s	remaining: 11.4s
494:	learn: 0.1215639	total: 11.1s	remaining: 11.3s
495:	learn: 0.1215251	total: 11.1s	remaining: 11.3s
496:	learn: 0.1214762	total: 11.1s	remaining: 11.3s
497:	learn: 0.1214297	total: 11.2s	remaining: 11.2s
498:	learn: 0.1213763	total: 11.2s	remaining: 11.2s
499:	learn: 0.1213210	total: 11.2s	remaining: 11.2s
500:	learn: 0.1212693	total: 11.2s	remaining: 11.2s
501:	learn: 0.1212104	total: 11.2s	remaining: 11.1s
502:	learn: 0.1211785	total: 11.3s	remaining: 11.1s
503:	learn: 0.1211011	total: 11.3s	remaining: 11.1s
504:	learn: 0.1210727	to

646:	learn: 0.1151994	total: 14.1s	remaining: 7.69s
647:	learn: 0.1151348	total: 14.1s	remaining: 7.67s
648:	learn: 0.1150643	total: 14.1s	remaining: 7.64s
649:	learn: 0.1150374	total: 14.2s	remaining: 7.62s
650:	learn: 0.1149995	total: 14.2s	remaining: 7.6s
651:	learn: 0.1149708	total: 14.2s	remaining: 7.58s
652:	learn: 0.1149365	total: 14.2s	remaining: 7.55s
653:	learn: 0.1148702	total: 14.2s	remaining: 7.53s
654:	learn: 0.1148366	total: 14.3s	remaining: 7.51s
655:	learn: 0.1148035	total: 14.3s	remaining: 7.48s
656:	learn: 0.1147659	total: 14.3s	remaining: 7.46s
657:	learn: 0.1146892	total: 14.3s	remaining: 7.44s
658:	learn: 0.1146531	total: 14.3s	remaining: 7.41s
659:	learn: 0.1145988	total: 14.3s	remaining: 7.39s
660:	learn: 0.1145666	total: 14.4s	remaining: 7.37s
661:	learn: 0.1145308	total: 14.4s	remaining: 7.34s
662:	learn: 0.1144925	total: 14.4s	remaining: 7.32s
663:	learn: 0.1144625	total: 14.4s	remaining: 7.29s
664:	learn: 0.1144317	total: 14.4s	remaining: 7.27s
665:	learn: 0

806:	learn: 0.1091530	total: 17.2s	remaining: 4.12s
807:	learn: 0.1091266	total: 17.2s	remaining: 4.1s
808:	learn: 0.1091008	total: 17.3s	remaining: 4.07s
809:	learn: 0.1090733	total: 17.3s	remaining: 4.05s
810:	learn: 0.1090477	total: 17.3s	remaining: 4.03s
811:	learn: 0.1090241	total: 17.3s	remaining: 4.01s
812:	learn: 0.1090047	total: 17.3s	remaining: 3.99s
813:	learn: 0.1089783	total: 17.3s	remaining: 3.96s
814:	learn: 0.1089407	total: 17.4s	remaining: 3.94s
815:	learn: 0.1089081	total: 17.4s	remaining: 3.92s
816:	learn: 0.1088806	total: 17.4s	remaining: 3.9s
817:	learn: 0.1088350	total: 17.4s	remaining: 3.88s
818:	learn: 0.1088063	total: 17.4s	remaining: 3.85s
819:	learn: 0.1087877	total: 17.5s	remaining: 3.83s
820:	learn: 0.1087287	total: 17.5s	remaining: 3.81s
821:	learn: 0.1087000	total: 17.5s	remaining: 3.79s
822:	learn: 0.1086724	total: 17.5s	remaining: 3.77s
823:	learn: 0.1086250	total: 17.5s	remaining: 3.75s
824:	learn: 0.1085908	total: 17.6s	remaining: 3.72s
825:	learn: 0.

968:	learn: 0.1039607	total: 20.4s	remaining: 652ms
969:	learn: 0.1039290	total: 20.4s	remaining: 631ms
970:	learn: 0.1038816	total: 20.4s	remaining: 610ms
971:	learn: 0.1038682	total: 20.4s	remaining: 588ms
972:	learn: 0.1038375	total: 20.4s	remaining: 567ms
973:	learn: 0.1038123	total: 20.5s	remaining: 546ms
974:	learn: 0.1037644	total: 20.5s	remaining: 525ms
975:	learn: 0.1037255	total: 20.5s	remaining: 504ms
976:	learn: 0.1036942	total: 20.5s	remaining: 483ms
977:	learn: 0.1036600	total: 20.5s	remaining: 462ms
978:	learn: 0.1036241	total: 20.6s	remaining: 441ms
979:	learn: 0.1035865	total: 20.6s	remaining: 420ms
980:	learn: 0.1035654	total: 20.6s	remaining: 399ms
981:	learn: 0.1035481	total: 20.6s	remaining: 378ms
982:	learn: 0.1035235	total: 20.6s	remaining: 357ms
983:	learn: 0.1035051	total: 20.6s	remaining: 336ms
984:	learn: 0.1034609	total: 20.7s	remaining: 315ms
985:	learn: 0.1034342	total: 20.7s	remaining: 294ms
986:	learn: 0.1034044	total: 20.7s	remaining: 273ms
987:	learn: 

In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat_ada_cc= GridSearchCV(estimator= cat_ada_cc, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat_ada_cc.fit(X_ada_cc, y_ada_cc)

print("Best Parameters:", gs_cat_ada_cc.best_params_)
print("Best Precision Score:", gs_cat_ada_cc.best_score_)

tuned_cat_ada_cc= gs_cat_ada_cc.best_estimator_
y_pred_tuned_cat_ada_cc= tuned_cat_ada_cc.predict(X_test)
y_pred_prob_tuned_cat_ada_cc= tuned_cat_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat_ada_cc, labels= tuned_cat_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_cat_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat_ada_cc))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.4865062	total: 19.4ms	remaining: 951ms
1:	learn: 0.3892331	total: 35.4ms	remaining: 849ms
2:	learn: 0.3315186	total: 56.8ms	remaining: 889ms
3:	learn: 0.2913179	total: 77.1ms	remaining: 887ms
4:	learn: 0.2594104	total: 94.2ms	remaining: 848ms
5:	learn: 0.2479737	total: 106ms	remaining: 780ms
6:	learn: 0.2400634	total: 118ms	remaining: 726ms
7:	learn: 0.2172919	total: 130ms	remaining: 680ms
8:	learn: 0.2118775	total: 140ms	remaining: 639ms
9:	learn: 0.2009033	total: 152ms	remaining: 608ms
10:	learn: 0.1981817	total: 162ms	remaining: 574ms
11:	learn: 0.1869309	total: 172ms	remaining: 546ms
12:	learn: 0.1785295	total: 183ms	remaining: 522ms
13:	learn: 0.1709442	total: 194ms	remaining: 500ms
14:	learn: 0.1695380	total: 204ms	remaining: 476ms
15:	learn: 0.1627443	total: 219ms	remaining: 465ms
16:	learn: 0.1621054	total: 232ms	remaining: 451ms
17:	learn: 0.1616194	total: 245ms	remaining: 436ms
18:	learn: 0.1569102	tota

--- Visualisation ---

In [35]:
print('Logistic Regression Precision:', precision_score(y_test, y_pred_lr_ada_cc, zero_division= 0))
print('Decision Tree Precision:', precision_score(y_test, y_pred_dt_ada_cc, zero_division= 0))
print('Tuned Decision Tree Precision:', precision_score(y_test, y_pred_tuned_dt_ada_cc, zero_division= 0))
print('KNeighborsClassifier Precision:', precision_score(y_test, y_pred_knn_ada_cc, zero_division= 0))
print('Tuned KNeighborsClassifier Precision:', precision_score(y_test, y_pred_tuned_knn_ada_cc, zero_division= 0))
print('GaussianNB Precision:', precision_score(y_test, y_pred_nb_ada_cc, zero_division= 0))
print('SVM Precision:', precision_score(y_test, y_pred_svc_ada_cc, zero_division= 0))
print('Random Forest Precision:', precision_score(y_test, y_pred_rf_ada_cc, zero_division= 0))
print('Tuned Random Forest Precision:', precision_score(y_test, y_pred_tuned_rf_ada_cc, zero_division= 0))
print('AdaBoost Precision:', precision_score(y_test, y_pred_ada_ada_cc, zero_division= 0))
print('Tuned AdaBoost Precision:', precision_score(y_test, y_pred_tuned_ada_ada_cc, zero_division= 0))
print('GradientBoosting Precision:', precision_score(y_test, y_pred_grb_ada_cc, zero_division= 0))
print('Tuned GradientBoosting Precision:', precision_score(y_test, y_pred_tuned_grb_ada_cc, zero_division= 0))
print('XGB Precision:', precision_score(y_test, y_pred_xgb_ada_cc, zero_division= 0))
print('Tuned XGB Precision:', precision_score(y_test, y_pred_tuned_xgb_ada_cc, zero_division= 0))
print('LGBM Precision:', precision_score(y_test, y_pred_lgm_ada_cc, zero_division= 0))
print('Tuned LGBM Precision:', precision_score(y_test, y_pred_tuned_lgm_ada_cc, zero_division= 0))
print('CatBoost Precision:', precision_score(y_test, y_pred_cat_ada_cc, zero_division= 0))
print('Tuned CatBoost Precision:', precision_score(y_test, y_pred_tuned_cat_ada_cc, zero_division= 0))

Logistic Regression Precision: 0.24387855044074436
Decision Tree Precision: 0.23302938196555217
Tuned Decision Tree Precision: 0.2944078947368421
KNeighborsClassifier Precision: 0.21696252465483234
Tuned KNeighborsClassifier Precision: 0.24643874643874644
GaussianNB Precision: 0.16666666666666666
SVM Precision: 0.15695067264573992
Random Forest Precision: 0.375
Tuned Random Forest Precision: 0.436
AdaBoost Precision: 0.389917695473251
Tuned AdaBoost Precision: 0.47649572649572647
GradientBoosting Precision: 0.41601255886970173
Tuned GradientBoosting Precision: 0.528125
XGB Precision: 0.4573643410852713
Tuned XGB Precision: 0.5154320987654321
LGBM Precision: 0.5466666666666666
Tuned LGBM Precision: 0.5424528301886793
CatBoost Precision: 0.5043103448275862
Tuned CatBoost Precision: 0.509090909090909


In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr_ada_cc, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt_ada_cc, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt_ada_cc, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn_ada_cc, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn_ada_cc, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb_ada_cc, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc_ada_cc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf_ada_cc, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf_ada_cc, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada_ada_cc, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada_ada_cc, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb_ada_cc, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb_ada_cc, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb_ada_cc, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb_ada_cc, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm_ada_cc, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm_ada_cc, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat_ada_cc, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat_ada_cc, zero_division= 0)
                  }

ada_cc_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
ada_cc_precision= ada_cc_precision.sort_values(by= 'Precision Score', ascending=False)
print(ada_cc_precision)

                                    Model  Precision Score
15                        LGBM Precision:         0.546667
16                  Tuned LGBM Precision:         0.542453
12      Tuned GradientBoosting Precision:         0.528125
14                   Tuned XGB Precision:         0.515432
18              Tuned CatBoost Precision:         0.509091
17                    CatBoost Precision:         0.504310
10              Tuned AdaBoost Precision:         0.476496
13                         XGB Precision:         0.457364
8          Tuned Random Forest Precision:         0.436000
11            GradientBoosting Precision:         0.416013
9                     AdaBoost Precision:         0.389918
7                Random Forest Precision:         0.375000
2          Tuned Decision Tree Precision:         0.294408
4   Tuned KNeighborsClassifier Precision:         0.246439
0          Logistic Regression Precision:         0.243879
1                Decision Tree Precision:         0.2330

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr_ada_cc),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt_ada_cc),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt_ada_cc),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn_ada_cc),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn_ada_cc),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb_ada_cc),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc_ada_cc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf_ada_cc),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf_ada_cc),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada_ada_cc),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada_ada_cc),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb_ada_cc),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb_ada_cc),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb_ada_cc),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb_ada_cc),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm_ada_cc),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm_ada_cc),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat_ada_cc),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat_ada_cc)
                  }

ada_cc_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
ada_cc_accuracy= ada_cc_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(ada_cc_accuracy)

                                   Model  Accuracy Score
15                        LGBM Accuracy:        0.905504
12      Tuned GradientBoosting Accuracy:        0.905186
16                  Tuned LGBM Accuracy:        0.905186
14                   Tuned XGB Accuracy:        0.904338
18              Tuned CatBoost Accuracy:        0.903701
17                    CatBoost Accuracy:        0.903489
13                         XGB Accuracy:        0.900944
10              Tuned AdaBoost Accuracy:        0.900944
8          Tuned Random Forest Accuracy:        0.899883
7                Random Forest Accuracy:        0.896065
11            GradientBoosting Accuracy:        0.891929
6                          SVM Accuracy:        0.887051
9                     AdaBoost Accuracy:        0.880581
2          Tuned Decision Tree Accuracy:        0.876763
4   Tuned KNeighborsClassifier Accuracy:        0.865521
1                Decision Tree Accuracy:        0.847386
3         KNeighborsClassifier 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr_ada_cc),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt_ada_cc),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt_ada_cc),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn_ada_cc),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn_ada_cc),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb_ada_cc),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc_ada_cc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf_ada_cc),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf_ada_cc),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada_ada_cc),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada_ada_cc),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb_ada_cc),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb_ada_cc),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb_ada_cc),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb_ada_cc),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm_ada_cc),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm_ada_cc),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat_ada_cc),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat_ada_cc)
                  }

ada_cc_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
ada_cc_auc= ada_cc_auc.sort_values(by= 'AUC Score', ascending=False)
print(ada_cc_auc)

                              Model  AUC Score
0          Logistic Regression AUC:   0.851377
10              Tuned AdaBoost AUC:   0.849442
12      Tuned GradientBoosting AUC:   0.848708
16                  Tuned LGBM AUC:   0.846574
14                   Tuned XGB AUC:   0.846119
15                        LGBM AUC:   0.845772
18              Tuned CatBoost AUC:   0.844795
11            GradientBoosting AUC:   0.843375
17                    CatBoost AUC:   0.840881
9                     AdaBoost AUC:   0.839905
13                         XGB AUC:   0.835753
8          Tuned Random Forest AUC:   0.831799
7                Random Forest AUC:   0.821288
5                   GaussianNB AUC:   0.801269
3         KNeighborsClassifier AUC:   0.739346
2          Tuned Decision Tree AUC:   0.701554
6                          SVM AUC:   0.699005
4   Tuned KNeighborsClassifier AUC:   0.620768
1                Decision Tree AUC:   0.581085
